# **BIRD SPECIES CLASSIFICATION USING MobileNet**
* The Dataset used in this notebook is [BIRDS 400 - SPECIES IMAGE CLASSIFICATION](https://www.kaggle.com/datasets/gpiosenka/100-bird-species),upload by Gerry on Kaggle.

In [2]:
# Importing Libraries and Packages
import logging
import matplotlib.pyplot as plt
import numpy as np
import os 
import pandas as pd
import PIL
import PIL.Image
import tensorflow as tf

from tensorflow.keras.layers import Layer

%matplotlib inline

logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [3]:
img_height = 128
img_width = 128
batch_size = 16

In [5]:
# Loading Data
img_path = '/kaggle/input/100-bird-species/train/ALBATROSS/002.jpg'
img = PIL.Image.open(img_path)
img

In [6]:
print ("image size: ", img.size)

In [7]:
data_dir = '/kaggle/input/100-bird-species/'

data_dir_train = os.path.join(data_dir, 'train')
data_dir_valid = os.path.join(data_dir, 'valid')
data_dir_test = os.path.join(data_dir, 'test')

In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir_train,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

valid_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir_valid,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir_test,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [10]:
labels = []
for x, y in train_ds:
    labels += y.numpy().tolist() 

In [11]:
print ("number of classes: ", len(set(labels)))

In [12]:
plt.hist(labels, bins=310)
plt.show()

In [13]:
# DATA PREPARATION AND PREPROCESSING
def normalize(img, label):
    return img / 255.0, label

In [14]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2)
])

In [15]:
train_dataset = (train_ds
                 .map(normalize)
                 .map(lambda x, y: (data_augmentation(x), y))
                 .prefetch(tf.data.AUTOTUNE))

valid_dataset = valid_ds.map(normalize)
test_dataset  = test_ds.map(normalize)

In [16]:
# MODELLING MOBILENET
def get_pretrained_mobilenet():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(128, 128, 3)),

        tf.keras.applications.mobilenet.MobileNet(input_shape=(128, 128, 3), weights='imagenet'),

        tf.keras.layers.Dense(768, activation='relu'),
        tf.keras.layers.Dense(768, activation='relu'),
        tf.keras.layers.Dense(315)
    ])
    model.get_layer(name='mobilenet_1.00_128').trainable = False
    
    return model

However, let's try build a simplifed version of MobileNet instead. We will use the **Depth-wise Separable Convolution**, which is the core feature of MobileNet.

Essentially, what Depth-wise separable convolution does is it simplifies the operations of standard convolution layers by having a depth-wise layer to do the multiplication and pointwise layer of size 1x1xN to do the addtion. As a result, it reduces the number of parameters drastically and can reach similar performance to the standard one.

**Note**: Below is not the actual MobileNet architecture.

In [17]:
class DepthwiseSeparableConvolution(Layer):

    def __init__(self, filter, kernel=3):
        super(DepthwiseSeparableConvolution,self).__init__()
        self.filter=filter
        self.kernel=kernel
    
    def build(self, input_shape):
        # Depthwise
        self.depthwise = tf.keras.layers.DepthwiseConv2D(self.kernel)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.relu1 = tf.keras.layers.ReLU()
        # Pointwise
        self.pointwise = tf.keras.layers.Conv2D(self.filter, 1)
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.relu2 = tf.keras.layers.ReLU()
        
    def call(self, inputs):
        x = self.depthwise(inputs)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pointwise(x)
        x = self.bn2(x)
        x = self.relu2(x)
        
        return x

In [18]:
def get_simple_mobilenet():
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(128, 128, 3)),

        # [!] Actual MobileNet has more layers and does not use MaxPool
        DepthwiseSeparableConvolution(128, 3),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        DepthwiseSeparableConvolution(256, 3),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        DepthwiseSeparableConvolution(512, 5),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        DepthwiseSeparableConvolution(1024, 5),
        
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(315)
    ])
    
    return model

In [19]:
# Just for comparison
def get_standard_conv():
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(128, 128, 3)),

        # [!] Actual MobileNet has more layers and does not use MaxPool
        tf.keras.layers.Conv2D(128, 3),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(256, 3),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(512, 5),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(1024, 5),

        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(315)
    ])
    
    return model

In [20]:
simple_mobilenet = get_simple_mobilenet()
simple_mobilenet.summary()

In [21]:
standard_conv = get_standard_conv()
standard_conv.summary()

In [24]:
# TRAINING
model = get_simple_mobilenet()
model.summary()

In [25]:
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy'])

In [26]:
checkpoint_path = "/checkpoints/simple_mobilenet"

model_history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5),
        tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True)
    ])

In [28]:
# MODEL EVALUATION:
model.evaluate(test_dataset)